STATISTICA AMONG RUMS WILL BE COLLECTED IN CSV FILE


METRIC TRACKING WILL BE DONE ON TOP OF IT

In [211]:
import pandas as pd
stat_file = pd.DataFrame({'model':[],
                  'model_params':[],
                  'opt_params':[],
                  'run_number':[],
                  'epoch':[],
                  'validation_or_train':[],
                  'accuracy':[],
                  'loss':[],
                'additional_info':[]})
stat_file

,model,model_params,opt_params,run_number,epoch,validation_or_train,accuracy,loss,additional_info


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import torch
import pandas as pd
from torch.utils.data import Dataset
import re 
import torch.nn as nn
class NewsDataset(Dataset):

    def __preprocess(self, data):
        data = data.fillna('absent')
        
        data.headline[data['headline'].apply(len) == 0] = 'absent'
        data.link[data['link'].apply(len) == 0] = 'absent'
        data.short_description[data['short_description'].apply(len) == 0] = 'absent'
        data.authors[data['authors'].apply(len) == 0] = 'absent'
        return data
        
    def __init__(self, path):
        """_summary_

        Args:
            path (str): path to .json file
        """
        j=pd.read_json(path, lines=True)
        j=self.__preprocess(j)
        
        link = j['link'].str.lower().replace(r'\b(www|http|https|com|html)\b',' ', regex=True)

        
        link = link.apply(lambda l: re.sub('[^a-z A-Z 0-9]+', ' ',l ))

        
        self.x = 'headline: ' +  j['headline'] + \
        ' ; short_description: ' + j['short_description'] + \
        ' ; authors: ' + j['authors'] + \
        ' ; link: ' + link

        
        self.y = j['category'].astype('category').cat.codes
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        """

        Args:
            idx (list): _description_

        Returns:
            str: single item
        """
        return {'x':self.x.iloc[idx],'y':self.y.iloc[idx]}
    


In [ ]:
import gensim
import numpy as np
from gensim.models import KeyedVectors
from gensim.utils import tokenize

In [165]:
embs = KeyedVectors.load_word2vec_format("/kaggle/input/google-glove/GoogleNews-vectors-negative300.bin", binary=True)
idx = embs.add_vector('unknowntoken',np.random.uniform(0,1,size=300))
idx = embs.add_vector('padtoken',np.random.uniform(0,1,size=300))

In [166]:
embs.key_to_index['padtoken']

3000001

In [167]:
len(embs.key_to_index)

3000002

In [168]:
embs[500].shape

(300,)

In [169]:
import gensim
import numpy as np
from gensim.models import KeyedVectors
from gensim.utils import tokenize

In [170]:
from torch.nn.utils.rnn import pad_sequence

In [171]:
class TokenizerAsWord2Vec():
    def __init__(self, keyed_vectors, padding=True, padtoken_idx=3000001):
        self.padtoken_idx=padtoken_idx
        self.keyed_vectors=keyed_vectors
        self.word_to_index = keyed_vectors.key_to_index
        self.splitter = tokenize

    def forward(self, x):
        """
        x - (B, L)
        """
        splitted= []
        for el in x:
            splitted.append(self.splitter(el))


        tokenized = []
        for s in splitted:
            s = list(s)
            for idx in range(len(s)):
                if s[idx] not in self.word_to_index:
                    s[idx]='unknowntoken'
            sentence=[]
            for el in s:
                
                sentence.append(self.word_to_index[el])
            tokenized.append(sentence)

        tensors=[]
        for sent in tokenized:
            sent = torch.tensor(sent)
            tensors.append(sent)
        tensors=pad_sequence(tensors, padding_value=self.padtoken_idx, batch_first=True)

        return tensors
                

In [172]:
tokenizer = TokenizerAsWord2Vec(embs, padtoken_idx=embs.key_to_index['padtoken'])

In [173]:
tokenizer.forward(["AASC dsvbhj! fv"])

tensor([[ 623846, 3000000, 3000000]])

In [174]:
class LSTMForClassificatoin(nn.Module):
    def __init__(self,vocab_size, emb_dim,bidirectional,  hidden_size_lstm, num_layers, hiddem_size_MLP, num_classes):
        super().__init__()
        self.bidirectional = bidirectional
        self.embs = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hidden_size_lstm, num_layers, batch_first=True,bidirectional=bidirectional)
        if bidirectional:
            self.linear_up = nn.Linear(hidden_size_lstm * 2, hiddem_size_MLP)
        else:
            self.linear_up = nn.Linear(hidden_size_lstm, hiddem_size_MLP)
        self.activation = nn.ReLU()
        self.linear_down = nn.Linear(hiddem_size_MLP, num_classes)

    def forward(self, x):
        """
        x - batched tokens
        Reurns:
            logits for classes
        """
        emb = self.embs(x)
        output, _ = self.lstm(emb)
        if self.bidirectional:
            lin1 = self.linear_up(output[:,-1,:] + output[:,0,:] )
        else:
            lin1 = self.linear_up(output[:,-1,:] )
        lin1 = self.activation(lin1)
        out = self.linear_down(lin1)

        return out

In [175]:
train_batch_size=32
eval_batch_size = 200
accumulation_steps = 150

train_test_split = [0.8, 0.2]
val_test_split = [0.5, 0.5]
epochs = 100

In [176]:

from torch.utils.data import DataLoader
ds = NewsDataset('/kaggle/input/news-category-dataset/News_Category_Dataset_v3.json')
from torch.utils.data import Dataset, DataLoader
generator = torch.Generator().manual_seed(42)
train_data, test_data = torch.utils.data.random_split(ds, train_test_split,generator=generator)
val_data, test_data = torch.utils.data.random_split(test_data, val_test_split,generator=generator)

train_dataloader = DataLoader(train_data, batch_size=train_batch_size,
                        shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_data, batch_size=eval_batch_size,
                        shuffle=True, num_workers=0)

<ipython-input-88-a90188d8aeda>:11: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data.headline[data['headline'].apply(len) == 0] = 'absent'
<ipython-input-88-a90188d8aeda>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [177]:
print('total samples:', len(ds))
print('train samples:', len(train_data))
print('test samples:', len(test_data))  

total samples: 209527
train samples: 167622
test samples: 20952


In [178]:
device = 'cuda:0'if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [179]:
def get_n_params(model):
    pp=0
    for p in list(model.named_parameters()):
        nn=1
        for s in list(p[1].size()):
            nn = nn*s
        pp += nn
        print(p[0])
        print(nn)
    return pp

In [ ]:
def evaluate(model, test_dataloader):
    with torch.no_grad():
            matches=0
            number_of_iters = 0
            for b_eval in test_dataloader:
                tokenized = tokenizer.forward(b_eval['x'])
                
                preds = model(tokenized.to(device))
                loss = criterion(preds, b_eval['y'].long().to(device))
                matches += torch.sum(torch.isclose(b_eval['y'].long().to(device), torch.argmax(preds, dim=1)))

            accuracy = matches / len(test_dataloader.dataset)
            print('accuracy: ', accuracy, 'loss',loss)
    return accuracy, loss

In [ ]:
LRs=[]
epoch_loss = []
epoch_acc_train=[]
epoch_acc_test=[]

In [ ]:
model_params = {'vocab_size':400002,
              'emb_dim':50,
              'hidden_size_lstm':256,
              'num_layers':3,
               'hiddem_size_MLP':1024,
               'num_classes':42,
               'bidirectional':True}
model = LSTMForClassificatoin(vocab_size=model_params['vocab_size'],
                          emb_dim=model_params['emb_dim'],
                          hidden_size_lstm=model_params['hidden_size_lstm'],
                          num_layers=model_params['num_layers'],
                          hiddem_size_MLP=model_params['hiddem_size_MLP'],
                          num_classes=model_params['num_classes'],
                          bidirectional=model_params['bidirectional'])


In [ ]:
model.embs.weight.requires_grad = True

In [ ]:
model.to(device)

In [ ]:
def insert_row(df, row):
    return pd.concat([df, row], ignore_index=True)
    

In [ ]:
run_number=0

In [ ]:
run_number+=1
def train(model, epochs, optimizer, criterion, scheduler, callback, callback_freq, additional_info):
    global stat_file
    for epoch_num in range(1,epochs):
        
        accuracies_test=  []
        accuracies_train=  []
        losses=[]
        
        ttl_loss=0
        train_iters=0
        for i, train_batch in enumerate(train_dataloader):
            
            # Tokenize and pad
            tokenized = tokenizer.forward(train_batch['x'])
            

            preds = model(tokenized.to(device))
            loss = criterion(preds, train_batch['y'].long().to(device))
            
            ttl_loss+=loss.item()
            train_iters+=1
            
            loss.backward()
            
            
            if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
                optimizer.step()                            # Now we can do an optimizer step
                model.zero_grad()
                
                print('loss', ttl_loss / train_iters)
                losses.append(ttl_loss / train_iters)
            
                
                ttl_loss=0
                train_iters=0
                
                
                train_accuracy = torch.sum(torch.isclose(train_batch['y'].long().to(device), torch.argmax(preds, dim=1))) / train_batch_size
                accuracies_train.append(train_accuracy)
                print('train accuracy: ',train_accuracy )
                
                row = {'model': model.__class__,
                  'model_params':str(model_params),
                  'opt_params':str(optimizer_params),
                  'run_number':run_number,
                  'epoch':epoch_num,
                  'validation_or_train':'train',
                  'accuracy':train_accuracy,
                  'loss':losses[-1],
                    'additional_info':additional_info}

                stat_file = insert_row(stat_file, pd.DataFrame(row, index=[len(stat_file)]))
                
            del preds
            del tokenized
            del loss
    
        scheduler.step()
        
        print('epoch ',str(epoch_num),'lr', scheduler.get_last_lr())
        LRs.append(scheduler.get_last_lr())

        if epoch_num % callback_freq == 0:
            val_acc, loss_val = callback(model, val_dataloader)
            epoch_acc_test.append(val_acc)
            print('accuracy: ', val_acc)
            row = {'model': model.__class__,
                  'model_params':str(model_params),
                  'opt_params':str(optimizer_params),
                  'run_number':run_number,
                  'epoch':epoch_num,
                  'validation_or_train':'validation',
                  'accuracy':val_acc,
                  'loss':loss_val,
                  'additional_info':additional_info}

            stat_file = insert_row(stat_file, pd.DataFrame(row, index=[len(stat_file)]))
        
        
        epoch_acc_train.append(accuracies_train)
        epoch_loss.append(losses)
        torch.save(model.state_dict(), 'weights'+str(epoch_num)+str(model.__class__)+'.pt')

    return stat_file

In [ ]:
optimizer_params = {'lr':0.00035, 'step_size':10, 'gamma':0.1}
callback_freq = 1
epochs = 20
optimizer = torch.optim.Adam(model.parameters(), lr=optimizer_params['lr'])
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                            step_size=optimizer_params['step_size'],
                                            gamma=optimizer_params['gamma'])


train(model=model,
      epochs=epochs,
      optimizer=optimizer,
      criterion=criterion,
      scheduler=scheduler,
      callback=evaluate,
      callback_freq=1,
     additional_info='glove pretrained ; not frozen')

In [ ]:
stat_file.to_csv('glove_lstm.csv')

In [ ]:
def get_parameter_ratios(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    ratio = trainable_params / total_params
    return trainable_params, total_params, ratio

# Usage
trainable_params, total_params, ratio = get_parameter_ratios(classification_bert)
print(f"Trainable Parameters: {trainable_params}")
print(f"Total Parameters: {total_params}")
print(f"Ratio: {ratio:.2%}")


In [ ]:
acc_test_epoch = [a.cpu() for a in epoch_acc_test]

In [ ]:
acc_train_epoch = []
for a in epoch_acc_train:
    acc_train_epoch.append([b.cpu() for b in a ])
acc_train_epoch = np.array(acc_train_epoch).mean(axis=1)



In [ ]:
acc_train_epoch

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1)
ax.plot(np.arange(len(acc_test_epoch)), acc_test_epoch, label='test')
ax.plot(np.arange(len(acc_train_epoch)), acc_train_epoch, label='train')
plt.legend()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(len(LRs)), LRs, label='test')
plt.legend()

In [ ]:
test_dataloader = DataLoader(test_data, batch_size=train_batch_size,
                        shuffle=True, num_workers=0)

In [ ]:
evaluate(classification_bert, test_dataloader)

In [180]:
category_to_index = {
    'ARTS': 0,
    'ARTS & CULTURE': 1,
    'BLACK VOICES': 2,
    'BUSINESS': 3,
    'COLLEGE': 4,
    'COMEDY': 5,
    'CRIME': 6,
    'CULTURE & ARTS': 7,
    'DIVORCE': 8,
    'EDUCATION': 9,
    'ENTERTAINMENT': 10,
    'ENVIRONMENT': 11,
    'FIFTY': 12,
    'FOOD & DRINK': 13,
    'GOOD NEWS': 14,
    'GREEN': 15,
    'HEALTHY LIVING': 16,
    'HOME & LIVING': 17,
    'IMPACT': 18,
    'LATINO VOICES': 19,
    'MEDIA': 20,
    'MONEY': 21,
    'PARENTING': 22,
    'PARENTS': 23,
    'POLITICS': 24,
    'QUEER VOICES': 25,
    'RELIGION': 26,
    'SCIENCE': 27,
    'SPORTS': 28,
    'STYLE': 29,
    'STYLE & BEAUTY': 30,
    'TASTE': 31,
    'TECH': 32,
    'THE WORLDPOST': 33,
    'TRAVEL': 34,
    'U.S. NEWS': 35,
    'WEDDINGS': 36,
    'WEIRD NEWS': 37,
    'WELLNESS': 38,
    'WOMEN': 39,
    'WORLD NEWS': 40,
    'WORLDPOST': 41
}


In [181]:
def convert_all_texts_to_table(tokenizer, embs, train_loader, test_loader, unk_token_idx, pad_token_idx):
    

    def prepare_one_batch_for_multinomial_NB(embs, batch, unk_token_idx, pad_token_idx):
        batch = tokenizer.forward(batch['x'])
        res = []
        for sent in batch:
            sent= sent.numpy()
            
            sent = sent[(sent!=unk_token_idx) & (sent != pad_token_idx)]
            sent_pres = embs.get_mean_vector(sent)
            
            res.append(sent_pres)
    
        return res

    train_features=[]
    y_train=[]
    for batch in train_loader:
        y_train.append([int(el) for el in batch['y']])
        train_features.append(prepare_one_batch_for_multinomial_NB(embs, batch, unk_token_idx, pad_token_idx))
        
    val_features=[]
    y_test=[]
    for batch in test_loader:
        y_test.append([int(el) for el in batch['y']])
        val_features.append(prepare_one_batch_for_multinomial_NB(embs, batch, unk_token_idx, pad_token_idx))

    return train_features, val_features, y_train, y_test

In [182]:
train_features, val_features, y_train, y_test = convert_all_texts_to_table(tokenizer,
                                                         embs,
                                                         train_dataloader,
                                                         val_dataloader,
                                                         embs.key_to_index['unknowntoken'],
                                                         embs.key_to_index['padtoken'])


In [183]:
train_features = np.array(train_features[:-1])
val_features = np.array(val_features[:-1])
y_train = np.array(y_train[:-1])
y_test = np.array(y_test[:-1])

In [184]:
train_features = train_features.reshape(-1,300)
val_features = val_features.reshape(-1,300)
y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)

In [185]:
print(val_features.shape)
print(y_test.shape)

(20800, 300)
(20800,)


In [207]:
from sklearn.metrics import f1_score
def train_NB(model, train_features, val_features, y_train, y_test):
    model.fit(train_features+2, y_train)
    preds = model.predict(val_features+2)
    print(f1_score(y_test, preds, average='macro'))
    return sum(preds == y_test) / len(preds)

In [208]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
acc = train_NB(nb, train_features, val_features, y_train, y_test)

0.006825135325959049


0.1673076923076923

In [212]:
row = {'model': nb.__class__,
                  'model_params':None,
                  'opt_params':None,
                  'run_number':None,
                  'epoch':None,
                  'validation_or_train':'validation',
                  'accuracy':acc,
                  'loss':None,
                  'additional_info':'multinomial NB - 0.006825135325959049 F1 score'}

stat_file = insert_row(stat_file, pd.DataFrame(row, index=[len(stat_file)]))

In [213]:
stat_file

,model,model_params,opt_params,run_number,epoch,validation_or_train,accuracy,loss,additional_info
0,<class 'sklearn.naive_bayes.MultinomialNB'>,None,None,None,None,validation,0.167308,None,multinomial NB - 0.006825135325959049 F1 score


In [190]:
stat_file.to_csv('glove-MNB.csv')